## **Installing Libraries**

In [1]:
!pip install python_speech_features

  Created wheel for python-speech-features: filename=python_speech_features-0.6-py3-none-any.whl size=5869 sha256=49de3fa8d7726b32250266c5f659ebb2db98f0ce04e2cc1ed81d63b32eb90ba7
  Stored in directory: /root/.cache/pip/wheels/5b/60/87/28af2605138deac93d162904df42b6fdda1dab9b8757c62aa3
Successfully built python-speech-features


In [2]:
#  os and argparse is done to read files from local folders
import os
import argparse

import numpy as np
from scipy.io import wavfile
import scipy.signal
from python_speech_features import mfcc
import matplotlib.pyplot as plt
import warnings
import random

# tesorflow is for the Deep learning model
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense ,LSTM , TimeDistributed
from tensorflow.keras.layers import Dropout
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

import IPython
import librosa
from scipy.signal import butter, lfilter
%matplotlib inline
import glob

In [3]:
#from google.colab import drive  # the sounds are stored in google drive 
#drive.mount('/content/gdrive')

In [4]:
# folder where files are stored
#cd /content/gdrive/'My Drive'/Colab Notebooks/data/training-a   

**about the dataset**

In [5]:
#############################################################  MFCC EXTRACTION   ##########################################################
# Variable assignment 
tt = 0
time_steps = 450
data_directory=os.getcwd()
print(data_directory)
#data_directory='/workspaces/python_env1/Creative_factory/the-circor-digiscope-phonocardiogram-dataset-1.0.3/AV'
#data_directory=dataset = '../dataset_heart_sound/AV'
data_directory=dataset = '../dataset_heart_sound/AV'
nfft = 1203 # Number of FFTs

# What diff ?
digit_directory = data_directory 

# To normalize the signal
def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0: 
        return v
    return v / norm
    
def process_directory():
    mfcc_features = list()
    """
    for filename in [x for x in os.listdir(data_directory) if x.endswith('.wav')]:
        # Read the .wav input file
        filepath = os.path.join(digit_directory, filename)
        sampling_freq, audio = wavfile.read(filepath)
        label="n.wav"


# open the .hea file of the same filename to get the label as normal /abnormal.
        st= data_directory +"/"+filename.split(".")[0]+".hea"
        with open(st,'r') as f:
            for line in f:
                for word in line.split():
                    if(word=="Abnormal"):
                        label="a.wav"
"""
    #dataset = [{'sampling_freq', 'audio': wavfile.read(path) , 'label': path.split('/' )[6] } for path in glob.glob("/workspaces/python_env1/Creative_factory/the-circor-digiscope-phonocardiogram-dataset-1.0.3/AV/**/*.wav")]
    for path in glob.glob("../dataset_heart_sound/AV/**/*.wav"):
        sampling_freq, audio = wavfile.read(path)
        label = path.split('/')[3]
            
# now we have the label stored in 'label' and the audio as 'audio' with sampling freq. as 'sampling_freq'.
        #audio1 = audio[dic[filename.split(".")[0]]:]
        #audio1 = SVDnoise(audio/32768)
        audio1 = audio
        temp = mfcc(audio1, sampling_freq, nfft=nfft)
        temp = temp[tt:tt+time_steps,:]
        mfcc_features.append({"label": label, "mfcc": temp })


        # mfcc features of this audio has been appended to the list 
    return mfcc_features

###########################   CREATING MFCC FEATURES   ############################
processed_files = list()
mfcc_features = process_directory()
random.shuffle(mfcc_features)

############   TRAINING DATA   ###########
size = (8*len(mfcc_features))/10
train_features = mfcc_features[0:int(size)]
test_list = mfcc_features[int(size+1):]
train_size = 0
for feature in train_features:
    train_size += 1
    processed_files.append({'label': feature["label"], 'feature': feature["mfcc"] })
# Train rnn for each MFCC and add to training set
x_train = np.zeros((train_size, time_steps ,13))
y_train = np.zeros((train_size))
i = 0
for processed_file in processed_files:
#       print(processed_file['label'])
#       print(processed_file['feature'].shape)
    x_train[i,:,:] = processed_file['feature']
    s = processed_file['label']
    if(s[0]=='a'):
        y_train[i]=1
    else:
        y_train[i]=0
    i += 1
normalize(x_train)

############   TESTING DATA   #############
test_files = list()
test_features = test_list
test_size = 0
for feature in test_features:
    test_size += 1
    test_files.append({'label': feature["label"], 'feature': feature["mfcc"] })
y_test = np.zeros((test_size))
x_test = np.zeros((test_size, time_steps ,13))
i = 0
for test_file in test_files:
    x_test[i,:,:] = test_file['feature']
    s = test_file['label']
#             print(s)
    if(s[0]=='a'):
        y_test[i]=1
    else:
        y_test[i]=0
    i += 1
normalize(x_test)
print('MFCC EXTRACTION DONE !')

/workspaces/Creative_factory_bit_2022/LSTM_demo
MFCC EXTRACTION DONE !


In [6]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)
print(y_test)
print(np.count_nonzero(y_test == 0))
print(np.count_nonzero(y_train == 0))
print(len(mfcc_features))
print(mfcc_features[1])


(629, 450, 13)
(157, 450, 13)
(629,)
(157,)
[1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 0. 1.
 1. 1. 0. 1. 1. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1.
 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1.
 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0.
 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 1.
 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
86
308
787
{'label': 'abnormal', 'mfcc': array([[ 17.25163153,  -7.47371584, -24.03596192, ...,  -4.92772153,
         -4.4076607 ,  -5.58898588],
       [ 13.21657076,  -9.4202569 ,  -9.92390891, ...,   3.89162192,
         13.99063658,  -4.71266614],
       [ 13.08747044,  -3.44597763,  -9.12435978, ...,  -4.39275704,
         18.12731561,  -7.2472483 ],
       ...,
       [ 12.86160226,  -6.19358918, -13.12099989, ..., -18.16718332,
          4.63782756,   4.927715  ],


## RNN MODEL
### Model Structure:
- visible layer or input layer : size of input = 13 (mfcc matrix has column size 13)
- hidden layer 1: LSTM layer
- hidden layer 2: LSTM layer
- 1 dense layer having activation function = "relu" (rectified linear)
- output layer : classification  

![download.png](attachment:download.png)  
<font size="2">*In the figure, the dense layer is missing, but this is to give a brief idea of how the data is flowing in the model structure.*</font>  
### Building the Model:
- loss function: to compute the loss (currently "mean squared error")
- optimizer function: adam
- metrics: accuracy
- **model.fit:** this function tries for the best possible fit of the model to the training data.
<br>
<font size="2"> <font color="brown"> The later part of the code was to try the model for different values of Dopout(lmabda) to calculate accuracy.</font>


In [7]:
dropouts = np.array([0.15])
accuracy = np.zeros(len(dropouts),dtype=float)

In [8]:
#########################  VARIABLES  ######################
cell_no = 13
Epoch_size = 50
Lambda = 0.029  # dropout variable for regularization
# No. of LSTM layers =2
# Cost func. = cosh

####################   MODEL STRUCTURE  ####################
visible=Input(shape=(None,13))
hidden11 = LSTM(cell_no,return_sequences=True)(visible)
hidden1=Dropout(Lambda)(hidden11)

hidden22 = LSTM(cell_no)(hidden1)
hidden2=Dropout(Lambda)(hidden22)

hidden33 = Dense(10, activation='relu')(hidden2)
hidden3 = Dropout(Lambda)(hidden33)
# hidden4 = TimeDistributed(Dense(1))

output = Dense(1, activation='sigmoid')(hidden3)
# output=Dropout(Lambda)(output1)

model = Model(inputs=visible, outputs=output)

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
##################   MODEL END ########################
count = 0
for dr in dropouts:
    total=0
    for ii in range(1):
        Lambda = dr
        print(dr)

        model.fit(x_train, y_train, epochs=Epoch_size, batch_size=10,verbose=1)
        predict=model.predict(x_test)
        y_pred=(predict>0.1)
        acc=model.evaluate(x_test,y_test,verbose=0)
        # print(predict)

        total += acc[1]
        print(acc)
    total /=1
    accuracy[count] = total
    count += 1

2022-09-03 22:41:24.557370: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:08:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-03 22:41:24.652917: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:08:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-03 22:41:24.653249: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:08:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-03 22:41:24.655372: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

0.15
Epoch 1/50


2022-09-03 22:41:32.546528: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8204


63/63 [==============================] - 13s 76ms/step - loss: 0.2503 - accuracy: 0.5151
Epoch 2/50
63/63 [==============================] - 5s 76ms/step - loss: 0.2468 - accuracy: 0.5517
Epoch 3/50
63/63 [==============================] - 5s 74ms/step - loss: 0.2417 - accuracy: 0.5930
Epoch 4/50
63/63 [==============================] - 5s 82ms/step - loss: 0.2356 - accuracy: 0.6137
Epoch 5/50
63/63 [==============================] - 5s 83ms/step - loss: 0.2303 - accuracy: 0.6328
Epoch 6/50
63/63 [==============================] - 5s 75ms/step - loss: 0.2253 - accuracy: 0.6534
Epoch 7/50
63/63 [==============================] - 5s 76ms/step - loss: 0.2152 - accuracy: 0.6709
Epoch 8/50
63/63 [==============================] - 5s 80ms/step - loss: 0.2067 - accuracy: 0.6852
Epoch 9/50
63/63 [==============================] - 5s 72ms/step - loss: 0.1983 - accuracy: 0.7218
Epoch 10/50
63/63 [==============================] - 5s 74ms/step - loss: 0.1856 - accuracy: 0.7361
Epoch 11/50
63/63 [

In [9]:

#Print f1, precision, and recall scores
print(precision_score(y_test, y_pred , average="macro"))
print(recall_score(y_test, y_pred , average="macro"))
print(f1_score(y_test, y_pred , average="macro"))
confusion = confusion_matrix(y_test,y_pred,labels=[0,1])
print(confusion)

0.5154458009618942
0.5136750736980019
0.4885149903088787
[[29 57]
 [22 49]]


In [10]:
accuracy

array([0.477707])